In [104]:
import requests # Импортируем библиотеку requests для запросов
from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup для парсинга
import bleach  # Импортируем библиотеку bleach для очистки текста от тэгов
import re      # Импортируем библиотеку для регулярных выражений
import pyautogui  # Импортируем библиотеку для интерактивного ввода
import pandas as pd
import os.path

In [105]:
# Если файлы таблиц существуют, считываем их
if os.path.isfile('./ingr_df.csv'):
    # Считываем таблицу ингредиентов из файла.
    ingr_df = pd.read_csv('./ingr_df.csv')
else:
    # Создаем таблицу таблицу ингредиентов, если файла с ней нет.
    ingr_df = pd.DataFrame(columns=['ingr_id', 'ingr_name'])

if os.path.isfile('./recipe_df.csv'):
    # Считываем таблицу рецептов из файла.
    recipe_df = pd.read_csv('./recipe_df.csv')
else:
    # Создаем таблицу таблицу рецептов, если файла с ней нет.
    recipe_df = pd.DataFrame(columns=['recipe_id', 'recipe_name', 'ingredients', 'quantity', 'text'])

In [106]:
# Создаем таблицу рецептов и таблицу ингредиентов. Только на первом проходе. 
# Потом они хранится где-то и пополняются уникальными рецептами и ингредиентами
#ingr_df = pd.DataFrame(columns=['ingr_id', 'ingr_name'])
#recipe_df = pd.DataFrame(columns=['recipe_id', 'recipe_name', 'ingredients', 'quantity', 'text'])
# recipe_df = pd.read_csv('./recipe_df.csv')
# ingr_df = pd.read_csv('./ingr_df.csv')

In [107]:
def add_ingr(ingr_name, ingr_df):
    """
    Функция добавляет ингредиент с уникальным именем в таблицу ингредиентов

        Args:
            ingr_name (string): наименование ингредиента
            ingr_df (dataframe): таблица ингредиентов
 
        Returns:
            ingr_df (dataframe): таблица ингредиентов с добавленным уникальным ингредиентом    
    """
    # dataframe текущего ингредиента
    curr_ingr = pd.DataFrame(columns=['ingr_id', 'ingr_name'], index=range(1))
    curr_ingr.ingr_name = ingr_name # наименование текущего ингредиента
    curr_ingr.ingr_id = ingr_df.ingr_id.count() # id текущего ингредиента
    # присоединение текущего ингредиента к исходному dataframe 
    ingr_df = pd.concat([ingr_df, curr_ingr], ignore_index=True)
    # удаление ингредиента, если такое наименование уже было в исходном dataframe  
    ingr_df.drop_duplicates(subset=['ingr_name'], inplace=True)
    return ingr_df

In [108]:
url = pyautogui.prompt(text="Выберите рецепт на сайте https://sbornik-recipes.ru и вставьте его адрес сюда", 
                                    title='Выбор рецепта' , 
                                    default='https://sbornik-recipes.ru/recipes/second/meat/81-kurica-s-kabachkami-v-duhovke.html')

In [109]:
# https://sbornik-recipes.ru/recipes/first/86-sup-iz-semgi-s-krasnoj-chechevicej.html
# https://sbornik-recipes.ru/recipes/salads/69-salat-s-fasolju-suharikami.html
# https://sbornik-recipes.ru/recipes/second/meat/8-nezhnye-lenivye-golubcy-na-skoruju-ruku.html
# https://sbornik-recipes.ru/recipes/desserts/80-tureckaja-pahlava.html

In [110]:
# Выполняем GET-запрос, содержимое ответа присваивается переменной response
response = requests.get(url, headers={
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'})
page = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер

In [111]:
# Название рецепта
recipe_name = page.title.text
recipe_name

'Нежные ленивые голубцы на скорую руку рецепт на 6 порции (Европейская кухня)'

In [112]:
# Текст рецепта
recipe_text = page.find(itemprop="recipeInstructions").text
recipe_text

'Представляем оригинальный домашний рецепт ленивых голубцов. Данное блюдо является диетическим, которое включает в себя овощную клетчатку и мясной белок. Такие нежные голубцы можно смело предлагать ребёнку и даже кушать во время диеты.Приготовление голубцов. Для этого нам понадобится любой фарш. Нужно учитывать качество мяса оно не должно быть жирным. Самый простой вариант - купить готовый фарш. Попробуйте поэкспериментировать с сочетанием мяса свинины и говядины 50/50. Либо можно просто обойтись куриным. Накладываем готовый фарш в глубокую миску, разбиваем яйца, по вкусу посыпаем солью и чёрным молотым перцем. Смешиваем рукой либо большой ложкой. Затем шинкуем капусту маленькими ломтиками. Для того чтобы капуста приобрела нежный вкус, нужно размять руками. Как альтернативу можно использовать толкачик для картофельного пюре. Когда капуста будет готова, то выложите её в наш фарш и перемешайте все ингредиенты.Когда наша масса будет готова, то мокрыми руками формируем форму для наших лени

In [113]:
# Ингредиенты
ingredients = str(page.find(itemprop="recipeIngredient"))
# Список ингредиентов с невыделенными подзаголовками и неудаленными тэгами
raw_ingr_list = ingredients.split("<br/>")
ingr_list=[]
for ingr_str in raw_ingr_list:
    isSubtitle = re.findall('<b>.*</b>', ingr_str)
    # Подзаголовки в списке ингредиентов
    if len(isSubtitle) != 0:
        # Выделение подзаголовков
        ingr_list.extend(re.split('<b>.*</b>',ingr_str))
    else:
        ingr_list.append(ingr_str)
# Словарь ингредиентов
ingr_dict = {}
for ingr_str in ingr_list:
    # Проверка наличия разделителя названия и меры ингредиента
    a = ingr_str.find('</i><i>')
    if a != -1:  # Если разделитель есть 
        # Разделение ингредиентов на название и меру
        k, v = ingr_str.split('</i><i>')
        # Очистка ингредиентов
        k = bleach.clean(k, tags=[], strip=True).replace('\n','').replace(':','')
        ingr_dict[k] = bleach.clean(v, tags=[], strip=True)
        # Добавление ингредиентов текущего рецепта в таблицу ингредиентов
        ingr_df = add_ingr(k, ingr_df)
print('Словарь ингредиентов текущего рецепта')
ingr_dict

Словарь ингредиентов текущего рецепта


{'Мясной фарш': '1 кг',
 'Белокочанная капуста': '400 гр.',
 'Куриное яйцо': '2 шт.',
 'Репчатый лук': '2-3 шт.',
 'Соль': 'по вкусу',
 'Перец чёрный молотый': 'по вкусу',
 'Сметана': 'по вкусу'}

In [114]:
# Текущий рецепт
recipe = pd.DataFrame(columns=['recipe_id', 'recipe_name', 'ingredients', 'quantity', 'text'])
recipe.ingredients = pd.Series(ingr_dict.keys())
recipe.quantity = pd.Series(ingr_dict.values())
recipe.recipe_name = recipe_name
recipe.text = recipe_text
if len(recipe_df) == 0: #если это первый рецепт в таблице
    recipe.recipe_id = 0
else:
    recipe.recipe_id = recipe_df.recipe_id.iloc[-1]+1
print('Таблица текущего рецепта')
recipe

Таблица текущего рецепта


,recipe_id,recipe_name,ingredients,quantity,text
0,9,Нежные ленивые голубцы на скорую руку рецепт н...,Мясной фарш,1 кг,Представляем оригинальный домашний рецепт лени...
1,9,Нежные ленивые голубцы на скорую руку рецепт н...,Белокочанная капуста,400 гр.,Представляем оригинальный домашний рецепт лени...
2,9,Нежные ленивые голубцы на скорую руку рецепт н...,Куриное яйцо,2 шт.,Представляем оригинальный домашний рецепт лени...
3,9,Нежные ленивые голубцы на скорую руку рецепт н...,Репчатый лук,2-3 шт.,Представляем оригинальный домашний рецепт лени...
4,9,Нежные ленивые голубцы на скорую руку рецепт н...,Соль,по вкусу,Представляем оригинальный домашний рецепт лени...
5,9,Нежные ленивые голубцы на скорую руку рецепт н...,Перец чёрный молотый,по вкусу,Представляем оригинальный домашний рецепт лени...
6,9,Нежные ленивые голубцы на скорую руку рецепт н...,Сметана,по вкусу,Представляем оригинальный домашний рецепт лени...


In [115]:
# Присоединение текущего рецепта к таблице рецептов
recipe_df = pd.concat([recipe_df, recipe], ignore_index=True)
# удаление дубликатов рецептов 
recipe_df.drop_duplicates(inplace=True, subset=['recipe_name', 'ingredients'])

In [116]:
# Запись в файлы
recipe_df.to_csv('./recipe_df.csv', index=False)
ingr_df.to_csv('./ingr_df.csv', index=False)

In [117]:
recipe_df

,recipe_id,recipe_name,ingredients,quantity,text
0,0,Курица с кабачками в духовке рецепт на 4 порци...,Куриное филе,500 гр.,"Куриное филе нарезать тонкими ломтиками, после..."
1,0,Курица с кабачками в духовке рецепт на 4 порци...,Кабачки,250 гр.,"Куриное филе нарезать тонкими ломтиками, после..."
2,0,Курица с кабачками в духовке рецепт на 4 порци...,Лук,180 гр.,"Куриное филе нарезать тонкими ломтиками, после..."
3,0,Курица с кабачками в духовке рецепт на 4 порци...,Сыр,150 гр.,"Куриное филе нарезать тонкими ломтиками, после..."
4,0,Курица с кабачками в духовке рецепт на 4 порци...,Чеснок,3 зуб.,"Куриное филе нарезать тонкими ломтиками, после..."
...,...,...,...,...,...
83,8,Рыбный рассольник рецепт на 4 порции (Европейс...,Лук,1 шт,"Картофель нарезать кубиками, опустить в кипящу..."
84,8,Рыбный рассольник рецепт на 4 порции (Европейс...,Консервы рыбные 1 банка,200 г,"Картофель нарезать кубиками, опустить в кипящу..."
85,8,Рыбный рассольник рецепт на 4 порции (Европейс...,"Соль, зелень укропа",по вкусу,"Картофель нарезать кубиками, опустить в кипящу..."
86,8,Рыбный рассольник рецепт на 4 порции (Европейс...,Перец чёрный горошком,3 шт,"Картофель нарезать кубиками, опустить в кипящу..."


In [118]:
ingr_df

,ingr_id,ingr_name
0,0,Куриное филе
1,1,Кабачки
2,2,Лук
3,3,Сыр
4,4,Чеснок
...,...,...
61,61,Укроп
62,62,Огурцы солёные
63,63,Консервы рыбные 1 банка
64,64,"Соль, зелень укропа"
